In [1]:
import requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [143]:
response = requests.get("https://www.basketball-reference.com/leagues/NBA_2018_games.html")

In [144]:
soup = BeautifulSoup(response.content, features= "html.parser")

In [145]:
col_name = []
for e in soup.find_all('tr')[0].find_all('th'):
    col_name.append(e.get_text())

In [146]:
col_name.remove('\xa0')
col_name.remove('\xa0')

In [147]:
col_name = col_name[:-1]

In [148]:
print(col_name)

['Date', 'Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS', 'Attend.']


In [149]:
col_name += ['Venue','Time of Game']

In [150]:
print(col_name)

['Date', 'Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS', 'Attend.', 'Venue', 'Time of Game']


In [151]:
df = pd.DataFrame(columns= col_name)
c = 1

for j in soup.find_all('div', {"class": "filter"})[0].find_all('a'):
    month_link = j.get('href')
    url = "https://www.basketball-reference.com"+ month_link
    print(url)
    response1 = requests.get(url)
    if response1.status_code == 200:
        soup1 = BeautifulSoup(response1.content, features= "html.parser")
        for k in range(len(soup1.find_all('tr'))):
            try:
                
                link = soup1.find_all("tr")[k].find_all('a')[-1].get('href')
                resp = requests.get("https://www.basketball-reference.com"+link)
                s = BeautifulSoup(resp.content, features= "html.parser")
                venue = s.find_all('div',{'class':"scorebox_meta"})[0].find_all('div')[1].get_text()
                time = re.findall('Time of Game:....',s.find('div').get_text().replace('\xa0',''))[0].split('e:')[-1]
                
                date = soup1.find_all('tr')[k].th.get_text()
                imd = []
                for i in soup1.find_all('tr')[k].find_all('td'):
                    imd.append(i.get_text())
                imd_1 = imd[:-1]  
                imd_1.remove('Box Score')
                imd_1.remove(imd_1[-2])
                df.loc[c-1] = [date] + imd_1 + [venue, time]
                c +=  1
            except ValueError:
                c += 1
            except IndexError:
                c +=1
            
    else:
        print("Unable to get response!. The status code of response of url {} is {}\n.".format(url, response1.status_code))

https://www.basketball-reference.com/leagues/NBA_2018_games-october.html
https://www.basketball-reference.com/leagues/NBA_2018_games-november.html
https://www.basketball-reference.com/leagues/NBA_2018_games-december.html
https://www.basketball-reference.com/leagues/NBA_2018_games-january.html
https://www.basketball-reference.com/leagues/NBA_2018_games-february.html
https://www.basketball-reference.com/leagues/NBA_2018_games-march.html
https://www.basketball-reference.com/leagues/NBA_2018_games-april.html
https://www.basketball-reference.com/leagues/NBA_2018_games-may.html
https://www.basketball-reference.com/leagues/NBA_2018_games-june.html


In [152]:
df.shape

(1311, 9)

In [153]:
df

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS,Attend.,Venue,Time of Game
1,"Tue, Oct 17, 2017",8:01p,Boston Celtics,99,Cleveland Cavaliers,102,"20,562","Quicken Loans Arena, Cleveland, Ohio",2:28
2,"Tue, Oct 17, 2017",10:30p,Houston Rockets,122,Golden State Warriors,121,"19,596","Oracle Arena, Oakland, California",2:28
3,"Wed, Oct 18, 2017",7:00p,Charlotte Hornets,90,Detroit Pistons,102,"20,491","Little Caesars Arena, Detroit, Michigan",2:04
4,"Wed, Oct 18, 2017",7:00p,Brooklyn Nets,131,Indiana Pacers,140,"15,008","Bankers Life Fieldhouse, Indianapolis, Indiana",2:19
5,"Wed, Oct 18, 2017",7:00p,Miami Heat,109,Orlando Magic,116,"18,846","Amway Center, Orlando, Florida",2:15
6,"Wed, Oct 18, 2017",7:00p,Philadelphia 76ers,115,Washington Wizards,120,"20,356","Capital One Arena, Washington, District of Col...",2:25
7,"Wed, Oct 18, 2017",7:30p,Milwaukee Bucks,108,Boston Celtics,100,"18,624","TD Garden, Boston, Massachusetts",2:15
8,"Wed, Oct 18, 2017",8:00p,New Orleans Pelicans,91,Memphis Grizzlies,103,"17,794","FedEx Forum, Memphis, Tennessee",2:22
9,"Wed, Oct 18, 2017",8:30p,Atlanta Hawks,117,Dallas Mavericks,111,"19,709","American Airlines Center, Dallas, Texas",2:15
10,"Wed, Oct 18, 2017",9:00p,Denver Nuggets,96,Utah Jazz,106,"17,588","Vivint Smart Home Arena, Salt Lake City, Utah",2:08


In [ ]:
# Give an year in place of year in NBA_year in below commad for which you have accumulated the data.
# df.to_excel("NBA_year.xlsx")